# Analyzing borrowers’ risk of defaulting

The project purpose is to prepare a report for a bank’s loan division. will need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

The report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

In [1]:
import pandas as pd # Loading all the libraries

In [2]:
# Load the data
credits = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [4]:
credits.shape

(21525, 12)

In [5]:
credits.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


from the current printed data above there seems to be some problems, there were obvious missing values in some columns (the 'days_employed' and 'total_income' that seems they are the same, people who deosnt work have no income). also the upper and lower cases should be all fixed.
and a negative values in the column 'days_employed'


In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


there are missing values in columns : days_employed and total_income.

In [7]:
credits[credits['days_employed'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


the missing values are symmetric because people doesnt work will not have any income (make sense)
will get the shape of both columns (the missing values -2174) with missing values to confirm that.

In [8]:
credits[(credits['days_employed'].isnull()) & (credits['total_income'].isnull())].shape

(2174, 12)

**Intermediate conclusion**

the number of rows and the number of missing values doesnt match.
need to check if these missing values with the two columns affect our data for getting a better result, need to check if the data in the age, income_type columns affected from these missing values.

In [9]:
missing_credits = credits[credits['days_employed'].isnull()]

In [10]:
missing_credits['income_type'].value_counts(normalize=True)# Checking distribution

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

**Possible reasons for missing values in data**
maybe because of business and retiree income types, there are many missing values there

In [11]:
credits['income_type'].value_counts()# Checking the distribution in the whole dataset

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

**looks like the missing values are random**

In [12]:
credits['income_type'].value_counts() / len(credits['income_type'])# Checking for other patterns

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
entrepreneur                   0.000093
unemployed                     0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64

will delete duplicates, replace missing values by the correct value should be in the cell, fix the upper and lower cases .

## Data transformation

In [13]:
credits['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [14]:
credits['education'] = credits['education'].str.lower() # fixing capital to small letters.#

In [15]:
credits['education'].value_counts()# Checking all the values in the column to make sure all fixed

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

[Check the data the `children` column]

In [16]:
credits['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

there are some problems with the data like: no one have -1 children or 20, its not normal need to check it.

In [17]:
credits['children'] = credits.drop(credits[credits['children'] == 20].index)
credits['children'] = credits.drop(credits[credits['children'] == -1].index)

In [18]:
credits['children'].value_counts()# Checking the `children` column again to make sure it's all fixed

0.0    14149
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

In [19]:
credits['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

will check how many negative rows i have in the whole dataset then decide if to delete them or change the minus value

In [20]:
credits[credits['days_employed'] < 0].shape[0]# Addressing the problematic values

15906

In [21]:
credits['days_employed'] = credits['days_employed'].abs()

In [22]:
credits['days_employed'].value_counts()# Checking the result

142.276217       1
1849.622944      1
886.253127       1
2539.534295      1
390574.985524    1
                ..
1394.302246      1
2325.720832      1
4086.407828      1
1259.497032      1
1636.419775      1
Name: days_employed, Length: 19351, dtype: int64

In [23]:
credits['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [24]:
credits['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

no one can be 0 years old, will delete it

In [25]:
credits.drop(credits[credits['dob_years'] == 0].index,inplace=True)

In [26]:
credits['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [27]:
credits['family_status'].value_counts()

married              12331
civil partnership     4156
unmarried             2797
divorced              1185
widow / widower        955
Name: family_status, dtype: int64

all good with the column 'familt_status' from what i see

In [28]:
credits['gender'].value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

In [29]:
credits.drop(credits[credits['gender'] == 'XNA'].index,inplace=True)

In [30]:
credits['gender'].value_counts()# Checking the result - make sure it's fixed

F    14164
M     7259
Name: gender, dtype: int64

In [31]:
credits['income_type'].value_counts()

employee                       11064
business                        5064
retiree                         3836
civil servant                   1453
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

all good with the column 'income_type' from what i see

In [32]:
credits.duplicated().sum()# Checking duplicates

71

** to have a better look at the data without repeating values on themselves and to have an accurate results when we need to calculate from our data**

In [33]:
credits = credits.drop_duplicates().reset_index(drop=True)

In [34]:
credits.duplicated().sum()

0

In [35]:
credits.shape

(21352, 12)

the data were 21525 rows and now it has 21230 after manipulations with the data, its less than 2% of our whole dataset, no duplicates

# Working with missing values

its good working with dictionaries so we can assign the keys a values when we need it in the code calling the key and not using a whole string to find specific values

In [36]:
education_d = credits[['education','education_id']]
education_d = education_d.drop_duplicates().reset_index(drop=True)

In [37]:
family_stat_d = credits[['family_status','family_status_id']]
family_stat_d = family_stat_d.drop_duplicates().reset_index(drop=True)

In [38]:
credits.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0.0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


### Restoring missing values in `total_income`

In [39]:
# function that calculates the age category
def age_calculate(age):
    try:
        if age <= 20:
            return '<20'
        elif 40 >= age >= 21:
            return('21 - 40')
        elif 65 >= age >= 41:
            return('41 - 65')
        else:
            return('>=66')
    except:
        return 0

In [40]:
age_calculate(15)# Test if the function works

'<20'

In [41]:
credits['age_calculate'] = credits['dob_years'].apply(age_calculate)

In [42]:
credits['age_calculate'].value_counts()

41 - 65    11202
21 - 40     9383
>=66         702
<20           65
Name: age_calculate, dtype: int64

In [43]:
years_nonan = credits[credits['age_calculate'].isnull() != True]
years_nonan.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_calculate
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 65
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21 - 40
2,0.0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21 - 40
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21 - 40
4,0.0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,41 - 65
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,21 - 40
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,41 - 65
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,41 - 65
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,21 - 40
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,41 - 65


In [44]:
years_nonan.pivot_table(index='income_type',columns= 'age_calculate',values='total_income', aggfunc='mean')

age_calculate,21 - 40,41 - 65,<20,>=66
income_type,,,,
business,31659.736404,33398.653730,20223.495136,33470.062000
civil servant,27073.519839,27636.337107,21099.632667,30992.299000
employee,25553.785402,26186.383363,19040.474088,26185.024870
entrepreneur,79866.103000,NaN,NaN,NaN
paternity / maternity leave,8612.661000,NaN,NaN,NaN
retiree,20980.397860,22409.712718,NaN,19666.105943
student,15712.260000,NaN,NaN,NaN
unemployed,9593.119000,32435.602000,NaN,NaN


In [45]:
years_nonan.pivot_table(index='income_type',columns= 'age_calculate',values='total_income',aggfunc='median')

age_calculate,21 - 40,41 - 65,<20,>=66
income_type,,,,
business,27305.901,28009.064,19648.2745,29314.4045
civil servant,24348.697,23646.526,12125.9860,26089.6870
employee,22713.764,22928.480,16356.9390,24643.1985
entrepreneur,79866.103,NaN,NaN,NaN
paternity / maternity leave,8612.661,NaN,NaN,NaN
retiree,18169.704,19475.237,NaN,17098.2030
student,15712.260,NaN,NaN,NaN
unemployed,9593.119,32435.602,NaN,NaN


In [46]:
years_nonan.pivot_table(index='income_type',columns= 'age_calculate',values='days_employed',aggfunc='median')

age_calculate,21 - 40,41 - 65,<20,>=66
income_type,,,,
business,1248.670968,2002.286795,542.867211,2318.709538
civil servant,2166.088523,3504.634243,509.969922,4137.331615
employee,1320.752994,2079.695512,868.770587,2830.361431
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,3296.759962,NaN,NaN,NaN
retiree,365336.560325,365001.329490,NaN,365934.432696
student,578.751554,NaN,NaN,NaN
unemployed,337524.466835,395302.838654,NaN,NaN


In [47]:
years_nonan.pivot_table(index='income_type',columns= 'age_calculate',values='days_employed',aggfunc='mean')

age_calculate,21 - 40,41 - 65,<20,>=66
income_type,,,,
business,1628.913011,2690.516766,515.985625,3725.387000
civil servant,2473.286031,4334.898360,600.437357,4145.742201
employee,1760.442986,2982.745709,782.125220,4092.413329
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,3296.759962,NaN,NaN,NaN
retiree,365321.022449,364885.813992,NaN,365640.450983
student,578.751554,NaN,NaN,NaN
unemployed,337524.466835,395302.838654,NaN,NaN


In [48]:
# a function that we will use for filling in missing values
pivot_income = years_nonan.pivot_table(index=['age_calculate','income_type'],
                                       columns='education',
                                       values='total_income',\
                                       aggfunc='median')
def median_income(y):
    education = y['education']
    age_calculate = y['age_calculate']
    income_type = y['income_type']
    try:
        return pivot_income[education][age_calculate][income_type]
    except:
        return 'error'

In [49]:
pivot_income['secondary education']['21 - 40']['business']# Check if it works

25417.268

In [50]:
credits['median_income'] = credits.apply(median_income,axis=1)

In [51]:
credits[credits['median_income'] == 'error']# Check if we got any errors

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_calculate,median_income
5907,0.0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,41 - 65,error


In [52]:
credits.drop(credits[credits['median_income'] == 'error'].index,inplace=True)

In [53]:
credits[credits['median_income'] == 'error']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_calculate,median_income


In [54]:
credits['total_income'] = credits['total_income'].fillna(credits.apply(median_income,axis=1))

In [55]:
credits['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21347    35966.698
21348    24959.969
21349    14347.610
21350    39054.888
21351    13127.587
Name: total_income, Length: 21351, dtype: float64

###  Restoring values in `days_employed`

In [56]:
years_nonan.pivot_table(index='income_type',columns='age_calculate',values='days_employed',aggfunc='median')

age_calculate,21 - 40,41 - 65,<20,>=66
income_type,,,,
business,1248.670968,2002.286795,542.867211,2318.709538
civil servant,2166.088523,3504.634243,509.969922,4137.331615
employee,1320.752994,2079.695512,868.770587,2830.361431
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,3296.759962,NaN,NaN,NaN
retiree,365336.560325,365001.329490,NaN,365934.432696
student,578.751554,NaN,NaN,NaN
unemployed,337524.466835,395302.838654,NaN,NaN


In [57]:
years_nonan.pivot_table(index='income_type',columns='age_calculate',values='days_employed',aggfunc='mean')

age_calculate,21 - 40,41 - 65,<20,>=66
income_type,,,,
business,1628.913011,2690.516766,515.985625,3725.387000
civil servant,2473.286031,4334.898360,600.437357,4145.742201
employee,1760.442986,2982.745709,782.125220,4092.413329
entrepreneur,520.848083,NaN,NaN,NaN
paternity / maternity leave,3296.759962,NaN,NaN,NaN
retiree,365321.022449,364885.813992,NaN,365640.450983
student,578.751554,NaN,NaN,NaN
unemployed,337524.466835,395302.838654,NaN,NaN


[Decide what you will use: means or medians. Explain why.]

In [58]:
mixed_days_income = credits.groupby('income_type')['days_employed'].median()
def fill_in_missing(income_type):
    try:
        return mixed_days_income[income_type]
    except:
        return 'error'

In [59]:
fill_in_missing('employee')# Check that the function works

1576.0676889889892

In [60]:
fill_in_missing('student')

578.7515535382181

In [61]:
credits['median_new'] = credits['income_type'].apply(fill_in_missing)

In [62]:
credits['income_type'].value_counts()

employee                       11029
business                        5057
retiree                         3809
civil servant                   1451
unemployed                         2
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [63]:
credits['days_employed'] = credits['days_employed'].fillna(fill_in_missing)# Replacing missing values

In [64]:
credits.drop(credits[credits['children'].isnull()].index,inplace=True)

In [65]:
credits.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_calculate       0
median_income       0
median_new          0
dtype: int64

## Categorization of data

In [66]:
credits['purpose'].value_counts()

wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    618
building a real estate                      617
buy real estate                             612
housing renovation                          602
buy residential real estate                 598
buying my own car                       

In [67]:
unique_purpose = credits['purpose'].unique()
print(unique_purpose)

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


In [68]:
## there are four main categories for borrowing a loan - wedding, housing, car and education
def purpose_category(y):
    print(y['purpose'])
    mapping = {'wedding':'wedding','estate':'house','property':'house',
           'hous':'house','car':'car','education':'education',
           'university':'education',
          'college':'education'}
    for key in mapping.keys():
        if key in y['purpose']:
            return mapping[key]
    return

In [69]:
credits['purpose_category'] = credits.apply(purpose_category,axis=1)

purchase of the house
car purchase
purchase of the house
supplementary education
to have a wedding
purchase of the house
housing transactions
education
having a wedding
purchase of the house for my family
buy real estate
buy commercial real estate
to have a wedding
car purchase
buy residential real estate
construction of own property
property
building a property
buying a second-hand car
buying my own car
property
car purchase
buying a second-hand car
to have a wedding
transactions with commercial real estate
buy real estate
education
construction of own property
construction of own property
building a real estate
transactions with commercial real estate
housing
having a wedding
purchase of the house
transactions with my real estate
having a wedding
cars
car purchase
education
to become educated
buy real estate
second-hand car purchase
getting an education
car purchase
buying my own car
to become educated
second-hand car purchase
having a wedding
construction of own property
car
cars
we

In [70]:
credits

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_calculate,median_income,median_new,purpose_category
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 65,27492.788,1576.067689,house
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21 - 40,21654.373,1576.067689,car
2,0.0,5623.42261,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21 - 40,21654.373,1576.067689,house
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21 - 40,21654.373,1576.067689,education
4,0.0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,41 - 65,18807.264,365176.336775,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1.0,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,41 - 65,25542.086,1548.009883,house
21348,0.0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,>=66,16867.066,365176.336775,car
21349,1.0,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,21 - 40,21654.373,1576.067689,house
21350,3.0,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,21 - 40,21654.373,1576.067689,car


In [71]:
print('hous' in 'housing')

print('hous' in 'house')

print('car' in 'to buy a car')

print('car' in 'housing')

True
True
True
False


In [72]:
credits.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_calculate,median_income,median_new,purpose_category
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 65,27492.788,1576.067689,house
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21 - 40,21654.373,1576.067689,car
2,0.0,5623.42261,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21 - 40,21654.373,1576.067689,house
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21 - 40,21654.373,1576.067689,education
4,0.0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,41 - 65,18807.264,365176.336775,wedding
5,0.0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,21 - 40,31391.403,1548.009883,house
6,0.0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,41 - 65,34707.5705,1548.009883,house
7,0.0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,41 - 65,22049.728,1576.067689,education
8,2.0,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,21 - 40,25865.55,1576.067689,wedding
9,0.0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,41 - 65,22049.728,1576.067689,house


In [73]:
credits['purpose_category'].describe()

count     20822
unique        4
top       house
freq      10702
Name: purpose_category, dtype: object

there were four main categories in the 'purpose' column, i categorized it as the code i wrote above with: wedding, house, education and car. 

In [74]:
# function for income ranges
def income_range(income):
    if (income > 0) and (income <= 15000):
        return 'small'
    if (income > 15000) and (income <= 30000):
        return 'average'
    if (income > 30000) and (income <= 80000):
        return 'above average income'
    if (income > 80000) and (income <= 200000):
        return 'high income'
    if (income > 200000):
        return 'very high income'

In [75]:
income_range(100000000)

'very high income'

In [77]:
credits['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21347    35966.698
21348    24959.969
21349    14347.610
21350    39054.888
21351    13127.587
Name: total_income, Length: 21229, dtype: float64

In [79]:
credits['total_income'].value_counts()

21654.373    389
22049.728    386
18807.264    266
25417.268    145
25542.086    142
            ... 
6264.532       1
27097.085      1
45484.109      1
27715.458      1
41428.916      1
Name: total_income, Length: 19170, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

there is a small correlation between having a children and paying back on time. 
means when people got more children will be a bit delayes with paying back approximately to those who doesnt have children.

In [80]:
credits.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_calculate,median_income,median_new,purpose_category
0,1.0,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41 - 65,27492.788,1576.067689,house
1,1.0,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,21 - 40,21654.373,1576.067689,car
2,0.0,5623.42261,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,21 - 40,21654.373,1576.067689,house
3,3.0,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,21 - 40,21654.373,1576.067689,education
4,0.0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,41 - 65,18807.264,365176.336775,wedding


In [81]:
wanted = credits.pivot_table(index='children',columns='debt',values='days_employed',aggfunc='count')# Check the children data and paying back on time
wanted['percentage'] = wanted[1] / (wanted[1] + wanted[0]*100)# Calculating default-rate based on the number of children
wanted.sort_values(by='percentage',ascending=True)

debt,0,1,percentage
children,,,
0.0,12962.0,1058.0,0.000816
3.0,301.0,27.0,0.000896
1.0,4351.0,441.0,0.001013
2.0,1845.0,194.0,0.001050
4.0,37.0,4.0,0.001080
5.0,9.0,NaN,NaN


**Is there a correlation between family status and paying back on time?**

there is a correlation but not a stron one too

In [82]:
wanted_family_pivot = credits.pivot_table(index='family_status',columns='debt',values='days_employed',aggfunc='count')# Check the family status data and paying back on time

wanted_family_pivot['percentage'] = wanted_family_pivot[1] / (wanted_family_pivot[1] + wanted_family_pivot[0]*100)# Calculating default-rate based on family status

wanted_family_pivot.sort_values(by='percentage',ascending=True)

debt,0,1,percentage
family_status,,,
widow / widower,884,62,0.000701
divorced,1095,84,0.000767
married,11289,923,0.000817
civil partnership,3729,383,0.001026
unmarried,2508,272,0.001083


**Conclusion**

there is a small connection between being married or not for being able to pay back a loan, but its not a big difference between both 

**Is there a correlation between income level and paying back on time?**

In [83]:
credits['income_range'] = credits['total_income'].apply(income_range)

In [84]:
wanted_income_pivot = credits.pivot_table(index='income_range',columns='debt',values='days_employed',aggfunc='count')# Check the income level data and paying back on time

wanted_income_pivot['percentage'] = wanted_income_pivot[1] / (wanted_income_pivot[1] + wanted_income_pivot[0]*100)# Calculating default-rate based on income level

wanted_income_pivot.sort_values(by='percentage', ascending=True)

debt,0,1,percentage
income_range,,,
high income,198,13,0.000656
above average income,5406,427,0.000789
small,3409,297,0.000870
average,10482,986,0.000940
very high income,10,1,0.000999


**How does credit purpose affect the default rate?**

In [85]:
wanted_puropse_pivot = credits.pivot_table(index='purpose_category',columns='debt',values='days_employed',aggfunc='count')

wanted_puropse_pivot['percentage'] = wanted_puropse_pivot[1] / (wanted_puropse_pivot[1] + wanted_puropse_pivot[0]*100)

wanted_puropse_pivot.sort_values(by='percentage',ascending=True)

debt,0,1,percentage
purpose_category,,,
house,9925,777,0.000782
wedding,2118,181,0.000854
education,3233,330,0.001020
car,3861,397,0.001027


In [86]:
credits['purpose_category'].value_counts()

house        10702
car           4258
education     3563
wedding       2299
Name: purpose_category, dtype: int64

# General Conclusion

there were some issues with the data, there are almost 20% of the values are missing of some reasons, maybe people who are retired and dont 
get any income from the goverments, students or any other reason. replaced all the missing values with the median 
value should be there. was about 71 duplicates that were removed. a few issued with the upped and lower cases 
all fixed.

we conclude that people who want to buy a house are the bigest slice of borrowers from our data, after that people who is buying a car then the others.